In [2]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import re
import scipy.stats as stats
from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'

sys.path.append(pathPrefix + '/juno/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 

In [ ]:
#FIRST FIX UP THE SIGNATURES

In [3]:
impactSignatures = pd.read_table(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/impactSignatureCalls_Nov20_2019.tsv')

In [ ]:
impactSignatures = mutationSigUtils.merge_signature_columns(impactSignatures, mode='Stratton', drop=True, smokingMerge=True, confidence=True, mean=True, prefix='mean_')

In [10]:
statsInfo = pd.read_table(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/mutations_TMB_and_MSI_stats.txt')


In [12]:
nMutDict = dict(zip(statsInfo['Tumor_Sample_Barcode'], statsInfo['n_mutations']))

In [20]:
tmbDict = dict(zip(statsInfo['Tumor_Sample_Barcode'], statsInfo['tmb']))

In [15]:
impactSignatures['Nmut'] = impactSignatures['Tumor_Sample_Barcode'].apply(lambda x: nMutDict[x] if x in nMutDict else None)

In [21]:
impactSignatures['Nmut_Mb'] = impactSignatures['Tumor_Sample_Barcode'].apply(lambda x: tmbDict[x] if x in tmbDict else None)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
impactSignatures = impactSignatures[impactSignatures['Nmut'].notnull()]

In [7]:
impactSignatures['dominantSignature'] = impactSignatures.apply(lambda row: mutationSigUtils.get_dominant_signature(row.to_dict(), cols=None, prefix='mean', notEnoughMuts= True), axis=1)

In [9]:
impactSignatures.to_csv(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/impactSignatureCalls_Nov20_2019.tsv', index=False, sep='\t')

In [10]:
#validates that the hypermutation cohort identified by the clustering fulfills two additional conditions:
#1. the majority signature in the hypermutated cases is distinct from the majority signature in the non-hypermutated cases
#2. the hypermutated cases represent <50% of the total cases
def validate_hypermutation_decomposition(hypermutationFileDir = pathPrefix + '/juno/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds/',
                                        signaturesFilePath = pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/impactSignatureCalls_Nov20_2019.tsv',
                                        minNormalCasesToCompare = 25 #dont change anything with the normal cases if there are fewer than this many signatures
                                        ):
    
    signaturesData = pd.read_table(signaturesFilePath)
    signaturesData = signaturesData[signaturesData['dominantSignature'] != 'insufficientMutBurden']
    dominantSignatureDict = dict(zip(signaturesData['Tumor_Sample_Barcode'], signaturesData['dominantSignature']))
    for f in os.listdir(hypermutationFileDir):
        print f
        fullFilePath = os.path.join(hypermutationFileDir, f)
        cancerTypeDataFrame = pd.read_table(fullFilePath)
        cancerTypeDataFrame['dominantSignature'] = cancerTypeDataFrame['Tumor_Sample_Barcode'].apply(lambda x: dominantSignatureDict[x] if x in dominantSignatureDict else None)
        
        allCases = set(cancerTypeDataFrame['Tumor_Sample_Barcode'])
        hypermutatedCases = set(cancerTypeDataFrame[cancerTypeDataFrame['hypermutantClassification'] == 'Hypermutated']['Tumor_Sample_Barcode'])
        normalCases = set(cancerTypeDataFrame[cancerTypeDataFrame['hypermutantClassification'] == 'Normal']['Tumor_Sample_Barcode'])
        
        hypermutatedDominantSignatures = signaturesData[signaturesData['Tumor_Sample_Barcode'].isin(hypermutatedCases)]['dominantSignature']
        normalDominantSignatures = signaturesData[signaturesData['Tumor_Sample_Barcode'].isin(normalCases)]['dominantSignature']
        
        if len(hypermutatedDominantSignatures) > 0 and len(normalDominantSignatures) > minNormalCasesToCompare:
            mostCommonHyperSig = Counter(hypermutatedDominantSignatures).most_common()[0][0]
            mostCommonNormalSig = Counter(normalDominantSignatures).most_common()[0][0]
            if mostCommonNormalSig == 'mean_MMR':
                mostCommonNormalSig = 'mean_1' #fix mmr dominant signature calls in non hypermutated which is probably a mistake
            if mostCommonNormalSig == mostCommonHyperSig:
                cancerTypeDataFrame['hypermutantClassification'] = cancerTypeDataFrame.apply(lambda row:
                            'highMutationBurden' if (row['dominantSignature'] == mostCommonNormalSig) and (row['hypermutantClassification'] == 'Hypermutated')
                            else row['hypermutantClassification'], axis=1)
        
        #RESAVE THE NEW HYPERMUTATION INFO
        print 'saving new version'
        cancerTypeDataFrame.to_csv(fullFilePath, index=False, sep='\t')


In [ ]:
#run this function to adjust the hypermutant decomposition as I want
validate_hypermutation_decomposition()


In [4]:










#ADJUSTING CODE, no need to worry about this delete eventually
#
##
#####
##########
sigsDf = pd.read_table(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/impactSignatureCalls_Nov20_2019_not_merged.tsv.bak')
statsInfo = pd.read_table(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/mutations_TMB_and_MSI_stats.txt')



In [6]:
tmbDict = dict(zip(statsInfo['Tumor_Sample_Barcode'], statsInfo['tmb']))
nMutDict = dict(zip(statsInfo['Tumor_Sample_Barcode'], statsInfo['n_mutations']))
sigsDf['Nmut'] = sigsDf['Tumor_Sample_Barcode'].apply(lambda x: nMutDict[x] if x in nMutDict else None)
sigsDf['Nmut_Mb'] = sigsDf['Tumor_Sample_Barcode'].apply(lambda x: tmbDict[x] if x in tmbDict else None)
sigsDf = sigsDf[sigsDf['Nmut'].notnull()]

In [13]:
renameDict = dict([('mean_' + str(i), 'Signature.' + str(i)) for i in range(1,31)])

In [16]:
sigsDf.to_csv(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/impactSignatureCalls_Nov20_2019_not_merged.tsv', index=False, sep='\t')